In [1]:
library(data.table)
library(glinternet)
library(pROC)
library(dplyr)
set.seed(1001)

Loaded glinternet 1.0.12


Type 'citation("pROC")' for a citation.


Attaching package: ‘pROC’


The following objects are masked from ‘package:stats’:

    cov, smooth, var



Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# Read dataset

In [2]:
meta  = fread('data/meta.csv')
prs   = fread('data/prs.csv')
pheno = fread('data/pheno.csv')

In [4]:
disease_codes <- c(
    diabetes = 'HC221', renal_failure = 'HC294', myocardial_infarction = 'HC326', asthma = 'HC382', 
    gall_stones = 'HC188', osteoarthritis = 'HC376', arthritis_nos = 'HC78',TTE_cystitis = 'HC1313', 
    TTE_chronic_renal_failure = 'HC1302',TTE_cellulitis = 'HC1139', TTE_cholelithiasis = 'HC1125')

# Print summary statistics of the cases

In [5]:
results <- list()
for (disease_name in names(disease_codes)) {
    disease_code <- disease_codes[disease_name]

    s_asian_cases_1       <- sum(pheno$population == "s_asian" & pheno[[disease_code]] == 2, na.rm = TRUE)
    white_british_cases_1 <- sum(pheno$population == "white_british" & pheno[[disease_code]] == 2, na.rm = TRUE)
    s_asian_cases_0       <- sum(pheno$population == "s_asian" & pheno[[disease_code]] == 1, na.rm = TRUE)
    white_british_cases_0 <- sum(pheno$population == "white_british" & pheno[[disease_code]] == 1, na.rm = TRUE)

    # Count SA cases with disease in each final_split category
    s_asian_cases_train <- sum(pheno$population == "s_asian" & pheno[[disease_code]] == 2 & pheno$final_split == "train", na.rm = TRUE)
    s_asian_cases_val   <- sum(pheno$population == "s_asian" & pheno[[disease_code]] == 2 & pheno$final_split == "val", na.rm = TRUE)
    s_asian_cases_test  <- sum(pheno$population == "s_asian" & pheno[[disease_code]] == 2 & pheno$final_split == "test", na.rm = TRUE)

    results[[disease_name]] <- c(SA_1 = s_asian_cases_1, WB_1 = white_british_cases_1, SA_0 = s_asian_cases_0, WB_0 = white_british_cases_0,
                                 SA_Train = s_asian_cases_train, SA_Val = s_asian_cases_val, SA_Test = s_asian_cases_test)
}

for (disease_name in names(results)) {
    disease_code <- disease_codes[disease_name]
    cat("Disease Name:", disease_name, "\nDisease Code:", disease_code, "\n")
    
    disease_matrix <- matrix(unlist(results[[disease_name]][1:4]), nrow = 2, byrow = TRUE, 
                             dimnames = list(c("SA", "WB"), c("Disease 1", "Disease 0")))
    print(disease_matrix)
    
    # Calculate and print the ratio of SA 1 to WB 1 cases
    ratio_sa_wb <- ifelse(disease_matrix[2,1] > 0, disease_matrix[1,1] / disease_matrix[2,1], "NA (No WB cases)")
    cat("Ratio of SA 1 to WB 1 cases:", ratio_sa_wb, "\n")

    # Print the number of SA cases with disease in each final_split category
    cat("SA cases with disease in Train:", results[[disease_name]]["SA_Train"], "\n")
    cat("SA cases with disease in Val:", results[[disease_name]]["SA_Val"], "\n")
    cat("SA cases with disease in Test:", results[[disease_name]]["SA_Test"], "\n\n")
}

Disease Name: diabetes 
Disease Code: HC221 
   Disease 1 Disease 0
SA       460      5479
WB      1451     75331
Ratio of SA 1 to WB 1 cases: 0.3170227 
SA cases with disease in Train: 0 
SA cases with disease in Val: 0 
SA cases with disease in Test: 0 

Disease Name: renal_failure 
Disease Code: HC294 
   Disease 1 Disease 0
SA        76      2644
WB      1835     78166
Ratio of SA 1 to WB 1 cases: 0.04141689 
SA cases with disease in Train: 0 
SA cases with disease in Val: 0 
SA cases with disease in Test: 0 

Disease Name: myocardial_infarction 
Disease Code: HC326 
   Disease 1 Disease 0
SA       151      3234
WB      1760     77576
Ratio of SA 1 to WB 1 cases: 0.08579545 
SA cases with disease in Train: 0 
SA cases with disease in Val: 0 
SA cases with disease in Test: 0 

Disease Name: asthma 
Disease Code: HC382 
   Disease 1 Disease 0
SA       285     10909
WB      1626     69901
Ratio of SA 1 to WB 1 cases: 0.1752768 
SA cases with disease in Train: 0 
SA cases with disease 

# Prepare dataset

In [8]:
prs$final_split   <- ifelse(is.na(prs$split_nonWB), prs$split, prs$split_nonWB)
pheno$final_split <- ifelse(is.na(pheno$split_nonWB), prs$split, pheno$split_nonWB)
meta$final_split  <- ifelse(is.na(pheno$split_nonWB), prs$split, pheno$split_nonWB)

In [9]:
global_pc_cols <- paste0("Global_PC", 1:40)
pc_cols        <- paste0("PC", 1:40)
drop_cols      <- c(global_pc_cols, pc_cols, 'split_nonWB', 'IID', 'population', 'age', 'age0', 'age1', 'age2', 'age3', 'sex', 'BMI', 'N_CNV', 'LEN_CNV', 'Array')

In [10]:
diseases <- disease_codes

In [11]:
drop_cols <- c(global_pc_cols, pc_cols, 'split','IID','age', 'age0', 'age1', 'age2', 'age3', 'sex', 'BMI', 'N_CNV', 'LEN_CNV', 'Array')

In [12]:
meta <- subset(meta, select = -which(names(meta) %in% drop_cols))

In [13]:
meta

population,Total_C,non_HDL_C,Remnant_C,VLDL_C,Clinical_LDL_C,LDL_C,HDL_C,Total_TG,VLDL_TG,⋯,M_HDL_C_pct,M_HDL_CE_pct,M_HDL_FC_pct,M_HDL_TG_pct,S_HDL_PL_pct,S_HDL_C_pct,S_HDL_CE_pct,S_HDL_FC_pct,S_HDL_TG_pct,final_split
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
white_british,1.687510729,1.8170812880,1.073787716,0.832070653,1.224198909,0.743176665,-0.12947124,1.66191485,1.40232003,⋯,-4.8070813,-4.504135575,-0.30268690,3.4846485,0.3322508,-2.4906363,-2.18190588,-0.31093711,2.159085118,test
white_british,-0.833291661,-1.3493873176,-0.655476210,-0.438618236,-1.199943274,-0.694034226,0.51609409,-0.58568322,-0.52716192,⋯,2.8009374,1.947288978,0.85299056,-1.7705246,2.3428036,-0.9435900,-1.43605119,0.48831546,-1.398387999,val
related,1.059777118,0.6750243429,0.366042102,0.158819042,0.458039834,0.308917086,0.38475308,-0.07803607,-0.15855809,⋯,0.9864535,-0.002902941,0.98981604,0.1336144,1.4875807,-1.5528009,-2.46058148,0.90707006,0.064723554,val
white_british,-0.820997386,-0.9206282485,-0.472597633,-0.252134660,-0.854446479,-0.447935864,0.09963055,-0.23886077,-0.25227562,⋯,0.4475054,0.264429714,0.18330576,-0.2043825,1.3219538,-1.3350482,-1.29650230,-0.03795047,0.007623726,train
DO_NOT_PASS_SQC,-1.183973764,-0.7992142806,-0.445388048,-0.127490903,-0.718281503,-0.353682355,-0.38481952,0.12182956,0.12409104,⋯,-2.1846542,-1.226840428,-0.95816050,1.3476682,-0.8639529,-0.2521782,0.06042538,-0.31138790,1.115139575,DO_NOT_PASS_SQC
white_british,3.196750084,3.4561344325,1.714055824,0.918165998,3.017776436,1.742201919,-0.25938305,0.78357061,0.63469429,⋯,-2.9482875,-3.410387698,0.46186707,3.1702444,-2.6815099,1.4602005,0.56710440,0.89340463,1.221991916,train
white_british,-0.239654374,0.1878943449,0.199597429,0.180443263,-0.004520687,-0.011582996,-0.42763868,0.31358961,0.16119679,⋯,-5.7119736,-5.173202745,-0.53951822,3.4500667,0.3022735,-2.4280978,-2.67032438,0.24255763,2.122951957,train
white_british,0.832385188,0.4259661692,0.090351851,-0.059521877,0.518107628,0.335100681,0.40632002,-0.38485583,-0.34633342,⋯,3.5392348,2.832190475,0.70717145,-1.9619964,-0.4298871,1.8219221,1.71707490,0.10646998,-1.393695529,train
white_british,-0.861022835,-0.8134380463,-0.375333741,-0.190140941,-0.744718554,-0.438029985,-0.04768502,-0.39950275,-0.33716519,⋯,0.4457610,0.541848963,-0.09661692,-0.4872765,0.9864947,-0.6260535,-0.48243556,-0.14444294,-0.362689546,train


# Split data

In [37]:
prepare_data <- function(populations, disease, pheno, meta, prs, use_prs) {
    
    set.seed(123)

    y <- subset(pheno, population %in% populations)
    
    if (use_prs == TRUE){
        X          <- subset(meta, population %in% populations)
        prs_column <- paste0('PRS_', disease)
        prs_subset <- subset(prs, population %in% populations, select = prs_column)
        X          <- cbind(X, prs_subset)
        }
    else {
        X <- subset(meta, population %in% populations)
    }
    
    print(X)
    # Ensure y only contains 1 or 2, and filter X accordingly
    valid_y_indices <- which(y[[disease]] %in% c(1, 2))
    y <- y[valid_y_indices,]
    X <- X[valid_y_indices,]

    X$population <- ifelse(X$population == "white_british", 0, ifelse(X$population == "s_asian", 1, NA))

    y_train_val <- subset(y, final_split %in% c('train', 'val'))[[disease]]
    y_test <- subset(y, final_split == 'test' & population == 's_asian')[[disease]]

    X_train_val <- subset(X, final_split %in% c('train', 'val'))
    X_test      <- subset(X, final_split == 'test' & population == 1)
    
    X_train_val$final_split <- NULL
    X_test$final_split      <- NULL

    # Impute missing numeric values in training and testing sets
    X_train_val <- X_train_val %>% mutate(across(where(is.numeric), ~ifelse(is.na(.), mean(., na.rm = TRUE), .)))
    X_test      <- X_test %>% mutate(across(where(is.numeric), ~ifelse(is.na(.), mean(., na.rm = TRUE), .)))

    y_train_val <- y_train_val - 1
    y_test      <- y_test - 1

    # Include all South Asian cases in training set
    sa_indices <- which(X_train_val$population == 1)

    # Count of South Asian cases without disease
    num_sa_without_disease <- sum(X_train_val$population == 1 & y_train_val == 0)

    # Sample up to an equal number of WB cases with and without disease
    wb_indices_with_disease    <- which(X_train_val$population == 0 & y_train_val == 1)
    wb_indices_without_disease <- which(X_train_val$population == 0 & y_train_val == 0)

    num_to_sample_wb_with    <- min(num_sa_without_disease, length(wb_indices_with_disease))
    num_to_sample_wb_without <- min(num_sa_without_disease, length(wb_indices_without_disease))

    sampled_wb_indices_with_disease    <- sample(wb_indices_with_disease, num_to_sample_wb_with)
    sampled_wb_indices_without_disease <- sample(wb_indices_without_disease, num_to_sample_wb_without)

    final_indices_train_val <- c(sa_indices, sampled_wb_indices_with_disease, sampled_wb_indices_without_disease)

    X_train <- X_train_val[final_indices_train_val, ]
    y_train <- y_train_val[final_indices_train_val]
    
    # Print out statistics
    combined_df          <- cbind(X_train, data.frame(y_train = y_train))
    
    combined_df$population <- ifelse(
        combined_df$population == 0, "WB", 
        ifelse(combined_df$population == 1, "SA", NA))

    combined_df$combined <- paste(combined_df$population, combined_df$y_train, sep = "-")
    value_counts <- table(combined_df$combined)
    print(value_counts)
    
    return(list(X_train = X_train, y_train = y_train, X_test = X_test, y_test = y_test))
}

# Train models

In [38]:
fit_models <- function(model_dir_path, disease, prepared_data, update_models) {
    
    cat('Disease : ', disease, '\n')
    model_path <- file.path(model_dir_path, paste0(disease, '.rds'))
    
    if (file.exists(model_path) && update_models == FALSE) {
        cat('Already fitted model for disease', disease, '\n\n')
        cv_fit = readRDS(file.path(model_dir_path, paste0(disease, '.rds')))
        return(cv_fit) }

    X_train <- prepared_data$X_train
    y_train <- prepared_data$y_train
    X_val   <- prepared_data$X_val
    y_val   <- prepared_data$y_val

    if (any(is.na(y_train)) || any(is.na(X_train))) {
        stop("NA values found in Y or X.")}
    
    num_cols <- ncol(X_train)
    numLevels <- c(2, rep(1, num_cols - 1))
    
    cat('Fitting model\n')
    flush.console()

    cv_fit <- glinternet.cv(
        X_train, y_train, numLevels, nFolds = 3, 
        family='binomial', interactionCandidates=1)    

    cat('Finished fitting model\n\n')
    flush.console()
    
    saveRDS(cv_fit, file.path(model_dir_path, paste0(disease, '.rds')))
    return (cv_fit)
}

# Fit models for WB-SA

In [39]:
populations <- list('white_british', 's_asian')
model_dir_path <- 'models/glinternet/WB_SA_metabolomics_PRS/'

# Fit models

In [40]:
aucs_with_prs  <- list()

for (disease in disease_codes) {
    prepared_data       <- prepare_data(populations, disease, pheno, meta, prs, use_prs=TRUE)
    system.time({cv_fit <- fit_models(model_dir_path, disease, prepared_data, update_models=FALSE)})
    
    i_1Std <- which(cv_fit$lambdaHat1Std == cv_fit$lambda)
    coefs <- coef(cv_fit$glinternetFit)[[i_1Std]]
    
    main_effects <- coefs$mainEffects$cont
    
    cat('Main effects:\n')
    for (effect in main_effects) {
        col_index <- effect
        print(colnames(prepared_data$X_train[, ..col_index]))
    }

    cat('\nInteractions:\n')
    
    if (length(coefs$interactions$catcont) > 0) {
    for (row in 1:nrow(coefs$interactions$catcont)) {
    
        var_idx <- (coefs$interactions$catcont[row, 2])
        print(colnames(prepared_data$X_train[, ..var_idx]))
    }
    }
    
    X_test <- prepared_data$X_test
    y_test <- prepared_data$y_test

    predictions <- as.vector(predict(cv_fit, X_test, type = "response"))

    roc_curve <- suppressMessages(roc(y_test, predictions, quietly = TRUE))
    auc_score <- auc(roc_curve)
    aucs_with_prs <- c(aucs_with_prs, auc_score)
    cat('\nAUC score: ', auc_score)
    cat('\n\n####################################################################################################')
    cat('\n####################################################################################################\n')

}

          population     Total_C   non_HDL_C   Remnant_C      VLDL_C
    1: white_british  1.68751073  1.81708129  1.07378772  0.83207065
    2: white_british -0.83329166 -1.34938732 -0.65547621 -0.43861824
    3: white_british -0.82099739 -0.92062825 -0.47259763 -0.25213466
    4: white_british  3.19675008  3.45613443  1.71405582  0.91816600
    5: white_british -0.23965437  0.18789434  0.19959743  0.18044326
   ---                                                              
82717: white_british  0.51746474  0.74499367  0.57617313  0.35656438
82718: white_british  0.43371654  0.44234425  0.12780314 -0.09341830
82719: white_british -0.51591114 -0.45034725 -0.18533347 -0.16179891
82720: white_british -0.00575579 -0.60172002 -0.38961791 -0.35649547
82721: white_british -0.34817818 -0.06855723  0.01497893 -0.02823024
       Clinical_LDL_C       LDL_C        HDL_C    Total_TG      VLDL_TG
    1:   1.2241989093  0.74317667 -0.129471244  1.66191485  1.402320034
    2:  -1.1999432736 -0.694

In [33]:
aucs_no_prs

[[1]]
[1] 0.7773852

[[2]]
[1] 0.7437673

[[3]]
[1] 0.622319

[[4]]
[1] 0.7491928

[[5]]
[1] 0.6030041

[[6]]
[1] 0.582146

[[7]]
[1] 0.5781915

[[8]]
[1] 0.7728723

[[9]]
[1] 0.724866

[[10]]
[1] 0.6288089

[[11]]
[1] 0.6983146

[[12]]
[1] 0.5068698

[[13]]
[1] 0.7857143

[[14]]
[1] 0.5545455

[[15]]
[1] 0.5927602

[[16]]
[1] 0.5937673

[[17]]
[1] 0.7577778

[[18]]
[1] 0.6236374

[[19]]
[1] 0.6684211

In [34]:
aucwith_prs

ERROR: Error in eval(expr, envir, enclos): object 'aucs_with_prs' not found
